In [14]:
from inaturalist import iNaturalistDataset, DATA_DIR
import torchvision.transforms as transforms
import json

import pandas as pd

In [15]:
all_transforms = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]
        ),
    ]
)

In [16]:
test = iNaturalistDataset(
    root="./data",
    train=False,
    download=False,
    classes=["Elasmobranchii"],  # inaturalist.FISH_CLASSES,
    transform=all_transforms,
)

In [17]:
test.annotations

,image_id,category_id,width,height,file_name,index,name,common_name,supercategory,kingdom,phylum,class,order,family,genus,specific_epithet,image_dir_name
0,150,4597,500,375,train/04597_Animalia_Chordata_Elasmobranchii_C...,4597,Triaenodon obesus,Whitetip Reef Shark,Animalia,Animalia,Chordata,Elasmobranchii,0,Carcharhinidae,Triaenodon,obesus,04597_Animalia_Chordata_Elasmobranchii_Carchar...
1,376,4597,500,229,train/04597_Animalia_Chordata_Elasmobranchii_C...,4597,Triaenodon obesus,Whitetip Reef Shark,Animalia,Animalia,Chordata,Elasmobranchii,0,Carcharhinidae,Triaenodon,obesus,04597_Animalia_Chordata_Elasmobranchii_Carchar...
2,2524,4597,500,375,train/04597_Animalia_Chordata_Elasmobranchii_C...,4597,Triaenodon obesus,Whitetip Reef Shark,Animalia,Animalia,Chordata,Elasmobranchii,0,Carcharhinidae,Triaenodon,obesus,04597_Animalia_Chordata_Elasmobranchii_Carchar...
3,2753,4601,500,292,train/04601_Animalia_Chordata_Elasmobranchii_M...,4601,Aetobatus narinari,Spotted Eagle Ray,Animalia,Animalia,Chordata,Elasmobranchii,4,Aetobatidae,Aetobatus,narinari,04601_Animalia_Chordata_Elasmobranchii_Mylioba...
4,2831,4609,500,316,train/04609_Animalia_Chordata_Elasmobranchii_O...,4609,Ginglymostoma cirratum,Nurse Shark,Animalia,Animalia,Chordata,Elasmobranchii,12,Ginglymostomatidae,Ginglymostoma,cirratum,04609_Animalia_Chordata_Elasmobranchii_Orectol...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3845,2683525,4597,500,397,train/04597_Animalia_Chordata_Elasmobranchii_C...,4597,Triaenodon obesus,Whitetip Reef Shark,Animalia,Animalia,Chordata,Elasmobranchii,0,Carcharhinidae,Triaenodon,obesus,04597_Animalia_Chordata_Elasmobranchii_Carchar...
3846,2683973,4597,500,375,train/04597_Animalia_Chordata_Elasmobranchii_C...,4597,Triaenodon obesus,Whitetip Reef Shark,Animalia,Animalia,Chordata,Elasmobranchii,0,Carcharhinidae,Triaenodon,obesus,04597_Animalia_Chordata_Elasmobranchii_Carchar...
3847,2684504,4601,500,334,train/04601_Animalia_Chordata_Elasmobranchii_M...,4601,Aetobatus narinari,Spotted Eagle Ray,Animalia,Animalia,Chordata,Elasmobranchii,4,Aetobatidae,Aetobatus,narinari,04601_Animalia_Chordata_Elasmobranchii_Mylioba...
3848,2685489,4603,500,375,train/04603_Animalia_Chordata_Elasmobranchii_M...,4603,Hypanus americanus,Southern Stingray,Animalia,Animalia,Chordata,Elasmobranchii,6,Dasyatidae,Hypanus,americanus,04603_Animalia_Chordata_Elasmobranchii_Mylioba...


In [18]:
train = False
classes = ["Elasmobranchii"]


with open(DATA_DIR / ("train.json" if train else "val.json")) as f:
    data = json.load(f)

In [19]:
categories = pd.DataFrame.from_dict(data["categories"])
categories = categories[categories["class"].isin(classes)]
categories = categories.reset_index()

cids = categories["id"]

In [20]:
categories["order"] = categories.index
categories = categories.set_index("id")
test.id_to_order = categories["order"].to_dict()

annotations = pd.DataFrame.from_dict(data["annotations"])
annotations = annotations.set_index(keys="id")
annotations = annotations[annotations["category_id"].isin(cids)]

In [21]:
images = pd.DataFrame.from_dict(data["images"]).set_index("id")

In [22]:
images[images.index.isin(annotations.index)]

,width,height,file_name,license,rights_holder,date,latitude,longitude,location_uncertainty
id,,,,,,,,,
2687016,500,333,val/04603_Animalia_Chordata_Elasmobranchii_Myl...,3,Mark Rosenstein,2012-07-27 15:05:06+00:00,19.38198,-81.32320,NaN
2687043,500,333,val/04599_Animalia_Chordata_Elasmobranchii_Het...,1,gfx,2012-11-24 00:00:00+00:00,-35.17136,138.46629,NaN
2687087,500,375,val/04605_Animalia_Chordata_Elasmobranchii_Myl...,1,rebeccafay,2009-08-09 20:18:32+00:00,37.80583,-122.44733,NaN
2687989,500,337,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,5,Kent Miller,2014-02-25 00:00:00+00:00,18.36250,-65.24331,30658.0
2689594,500,470,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,1,107120830535687523870,2015-01-18 16:00:00+00:00,11.23926,-60.53201,31142.0
...,...,...,...,...,...,...,...,...,...
2785600,500,375,val/04609_Animalia_Chordata_Elasmobranchii_Ore...,2,autumndrose,2016-06-17 19:56:00+00:00,17.79526,-88.00831,3015.0
2786163,500,375,val/04603_Animalia_Chordata_Elasmobranchii_Myl...,1,jacobjensen11,2016-11-26 18:56:00+00:00,19.32217,-81.24087,18370.0
2786229,500,375,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,1,Mike Jones,2013-01-14 15:23:00+00:00,20.34713,-87.20541,30490.0


In [23]:
annotations["image_id"].value_counts()

image_id
2687016    1
2687043    1
2687087    1
2687989    1
2689594    1
          ..
2785600    1
2786163    1
2786229    1
2786408    1
2786541    1
Name: count, Length: 160, dtype: int64

In [24]:
annotations.set_index("image_id").join(images)

,category_id,width,height,file_name,license,rights_holder,date,latitude,longitude,location_uncertainty
image_id,,,,,,,,,,
2687016,4603,500,333,val/04603_Animalia_Chordata_Elasmobranchii_Myl...,3,Mark Rosenstein,2012-07-27 15:05:06+00:00,19.38198,-81.32320,NaN
2687043,4599,500,333,val/04599_Animalia_Chordata_Elasmobranchii_Het...,1,gfx,2012-11-24 00:00:00+00:00,-35.17136,138.46629,NaN
2687087,4605,500,375,val/04605_Animalia_Chordata_Elasmobranchii_Myl...,1,rebeccafay,2009-08-09 20:18:32+00:00,37.80583,-122.44733,NaN
2687989,4601,500,337,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,5,Kent Miller,2014-02-25 00:00:00+00:00,18.36250,-65.24331,30658.0
2689594,4601,500,470,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,1,107120830535687523870,2015-01-18 16:00:00+00:00,11.23926,-60.53201,31142.0
...,...,...,...,...,...,...,...,...,...,...
2785600,4609,500,375,val/04609_Animalia_Chordata_Elasmobranchii_Ore...,2,autumndrose,2016-06-17 19:56:00+00:00,17.79526,-88.00831,3015.0
2786163,4603,500,375,val/04603_Animalia_Chordata_Elasmobranchii_Myl...,1,jacobjensen11,2016-11-26 18:56:00+00:00,19.32217,-81.24087,18370.0
2786229,4601,500,375,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,1,Mike Jones,2013-01-14 15:23:00+00:00,20.34713,-87.20541,30490.0


In [25]:
annotations = annotations.set_index("image_id").join(images)
annotations = annotations[["category_id", "width", "height", "file_name"]]

annotations = annotations.join(
    categories,
    on="category_id",
    how="left",
    lsuffix="_left",
    rsuffix="_right",
)

annotations = annotations.reset_index()

In [26]:
annotations

,image_id,category_id,width,height,file_name,index,name,common_name,supercategory,kingdom,phylum,class,order,family,genus,specific_epithet,image_dir_name
0,2687016,4603,500,333,val/04603_Animalia_Chordata_Elasmobranchii_Myl...,4603,Hypanus americanus,Southern Stingray,Animalia,Animalia,Chordata,Elasmobranchii,6,Dasyatidae,Hypanus,americanus,04603_Animalia_Chordata_Elasmobranchii_Mylioba...
1,2687043,4599,500,333,val/04599_Animalia_Chordata_Elasmobranchii_Het...,4599,Heterodontus portusjacksoni,Port Jackson Shark,Animalia,Animalia,Chordata,Elasmobranchii,2,Heterodontidae,Heterodontus,portusjacksoni,04599_Animalia_Chordata_Elasmobranchii_Heterod...
2,2687087,4605,500,375,val/04605_Animalia_Chordata_Elasmobranchii_Myl...,4605,Myliobatis californica,Bat Ray,Animalia,Animalia,Chordata,Elasmobranchii,8,Myliobatidae,Myliobatis,californica,04605_Animalia_Chordata_Elasmobranchii_Mylioba...
3,2687989,4601,500,337,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,4601,Aetobatus narinari,Spotted Eagle Ray,Animalia,Animalia,Chordata,Elasmobranchii,4,Aetobatidae,Aetobatus,narinari,04601_Animalia_Chordata_Elasmobranchii_Mylioba...
4,2689594,4601,500,470,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,4601,Aetobatus narinari,Spotted Eagle Ray,Animalia,Animalia,Chordata,Elasmobranchii,4,Aetobatidae,Aetobatus,narinari,04601_Animalia_Chordata_Elasmobranchii_Mylioba...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2785600,4609,500,375,val/04609_Animalia_Chordata_Elasmobranchii_Ore...,4609,Ginglymostoma cirratum,Nurse Shark,Animalia,Animalia,Chordata,Elasmobranchii,12,Ginglymostomatidae,Ginglymostoma,cirratum,04609_Animalia_Chordata_Elasmobranchii_Orectol...
156,2786163,4603,500,375,val/04603_Animalia_Chordata_Elasmobranchii_Myl...,4603,Hypanus americanus,Southern Stingray,Animalia,Animalia,Chordata,Elasmobranchii,6,Dasyatidae,Hypanus,americanus,04603_Animalia_Chordata_Elasmobranchii_Mylioba...
157,2786229,4601,500,375,val/04601_Animalia_Chordata_Elasmobranchii_Myl...,4601,Aetobatus narinari,Spotted Eagle Ray,Animalia,Animalia,Chordata,Elasmobranchii,4,Aetobatidae,Aetobatus,narinari,04601_Animalia_Chordata_Elasmobranchii_Mylioba...
158,2786408,4609,500,281,val/04609_Animalia_Chordata_Elasmobranchii_Ore...,4609,Ginglymostoma cirratum,Nurse Shark,Animalia,Animalia,Chordata,Elasmobranchii,12,Ginglymostomatidae,Ginglymostoma,cirratum,04609_Animalia_Chordata_Elasmobranchii_Orectol...


In [28]:
test.annotations["common_name"].unique()

array(['Whitetip Reef Shark', 'Spotted Eagle Ray', 'Nurse Shark',
       'Southern Stingray', 'Bluespotted Ribbontail Ray',
       "Haller's Round Ray", 'Bat Ray', 'Smooth Stingray',
       'Leopard Shark', 'New Zealand Eagle Ray', 'Grey Nurse Shark',
       'Spotted Wobbegong', 'Port Jackson Shark', 'Banded Wobbegong',
       'Banjo Shark', 'Common Stingaree'], dtype=object)